# database
> Using Pydantic's BaseSettings object to manage SQLAlchemy Database engines.

In [ ]:
#| default_exp database

In [ ]:
#| exporti

from humble_database.utils import delegates
import sqlalchemy
from sqlalchemy import create_engine, URL, Engine
from sqlalchemy.orm import Session
from pydantic import SecretStr
from pydantic_settings import BaseSettings, SettingsConfigDict
from typing import Optional,Union
from abc import ABC, abstractproperty,abstractmethod
from contextlib import contextmanager
import pandas as pd
from sqlalchemy import text

In [ ]:
#| hide 

from nbdev.showdoc import show_doc
import os

## SQL Alchemy Connection

In [ ]:
show_doc(URL.create)

---

### URL.create

>      URL.create (drivername:str, username:Optional[str]=None,
>                  password:Optional[str]=None, host:Optional[str]=None,
>                  port:Optional[int]=None, database:Optional[str]=None, query:M
>                  apping[str,Union[Sequence[str],str]]=immutabledict({}))

Create a new :class:`_engine.URL` object.

.. seealso::

    :ref:`database_urls`

:param drivername: the name of the database backend. This name will
  correspond to a module in sqlalchemy/databases or a third party
  plug-in.
:param username: The user name.
:param password: database password.  Is typically a string, but may
  also be an object that can be stringified with ``str()``.

  .. note::  A password-producing object will be stringified only
     **once** per :class:`_engine.Engine` object.  For dynamic password
     generation per connect, see :ref:`engines_dynamic_tokens`.

:param host: The name of the host.
:param port: The port number.
:param database: The database name.
:param query: A dictionary of string keys to string values to be passed
  to the dialect and/or the DBAPI upon connect.   To specify non-string
  parameters to a Python DBAPI directly, use the
  :paramref:`_sa.create_engine.connect_args` parameter to
  :func:`_sa.create_engine`.   See also
  :attr:`_engine.URL.normalized_query` for a dictionary that is
  consistently string->list of string.
:return: new :class:`_engine.URL` object.

.. versionadded:: 1.4

    The :class:`_engine.URL` object is now an **immutable named
    tuple**.  In addition, the ``query`` dictionary is also immutable.
    To create a URL, use the :func:`_engine.url.make_url` or
    :meth:`_engine.URL.create` function/ method.  To modify a
    :class:`_engine.URL`, use the :meth:`_engine.URL.set` and
    :meth:`_engine.URL.update_query` methods.

In [ ]:
#| export 

class DatabaseSettings(BaseSettings):
    drivername:str
    username: Optional[str]=None
    password: Optional[SecretStr]=None
    host: Optional[str]=None
    port: Optional[int]=None
    database: Optional[str]=None
    query: dict[str,str]={}
    


In [ ]:
settings = DatabaseSettings(
    drivername='sqlite',
    database='test.db',
)
settings

DatabaseSettings(drivername='sqlite', username=None, password=None, host=None, port=None, database='test.db', query={})

In [ ]:
url = URL.create(
    **settings.model_dump()
)
url

sqlite:///test.db

In [ ]:
engine = create_engine(url)
engine

Engine(sqlite:///test.db)

In [ ]:
#| exporti 

class AbstractDatabaseClass(ABC):

    """
    Abstract Base Class for all Database Connections.

    <br><br>
    From [SQLAlchemy docs](https://docs.sqlalchemy.org/en/13/core/engines.html):
    > The Engine is the starting point for any SQLAlchemy application. It’s “home base” for the actual database and its DBAPI.
    > An Engine references both a Dialect and a Pool, which together interpret the DBAPI’s module functions as well as the behavior of the database <br><br>
    > Pool object which will establish a DBAPI connection at localhost:5432 when a connection request is first received
    > - Note that the Engine and its underlying Pool do **not** establish the first actual DBAPI connection until the Engine.connect() method is called, or an operation which is dependent on this method such as Engine.execute() is invoked.
    > - In this way, Engine and Pool can be said to have a lazy initialization behavior.
    >
    > The Engine, once created, can either be used directly to interact with the database, or can be passed to a Session object to work with the ORM.

    """

    def query_to_records(
        self,
        query_string:str,
    ):
        with self._engine.connect() as conn:
            results = [row for row in conn.execute(text(query_string)).mappings()]
        return results

    @delegates(pd.read_sql_query)
    def query_to_df(
        self,
        query_string,
        **kwargs
    ):
        f"""{pd.read_sql_query.__doc__}"""
        with self._engine.connect() as conn:
            df = pd.read_sql_query(query_string,conn,**kwargs)
        return df

    @contextmanager
    def session_scope(self,bind=None,**kwargs):
        """Provide a transactional scope around a series of operations."""

        session = Session(bind=self._engine,**kwargs)
        try:
            yield session
            session.commit()
        except:
            session.rollback()
            raise
        finally:
            session.close()


In [ ]:
#| export 

class Database(DatabaseSettings,AbstractDatabaseClass):
    
    _engine:Engine = None
    _engine_url:URL = None

    def __init__(
        self,
        **kwargs
    ):
        # settings __init__
        super().__init__(**kwargs)
        if hasattr(self.password,'get_secret_value'):
            
            password = self.password.get_secret_value()
            print(password)
        else:
            password = self.password
        url = URL.create(
            drivername=self.drivername,
            username=self.username,
            password=password,
            host=self.host,
            port=self.port,
            database=self.database,
            query=self.query
        )
        self._engine_url=url
        self._engine=create_engine(url)

    
    model_config = SettingsConfigDict(
        #allows for attributes of `database settings` to be set as defaults in subclasses without type annotation
        ignored_types=(int,str,dict),
        arbitrary_types_allowed=True
    )
    

In [ ]:
database = Database(drivername='sqlite',database='test.db')
database

Database(drivername='sqlite', username=None, password=None, host=None, port=None, database='test.db', query={})

In [ ]:
db = Database(drivername='sqlite',database='test.db')

users = pd.DataFrame({
    'id':[1,2,3],
    'user':['larry','moe','curly']
})

users.to_sql('users',db._engine,if_exists='replace',index=False)

queried = db.query_to_df("select * from users")

assert queried.equals(users)
os.remove('test.db')

In [ ]:
#| hide

!nbdev_export